In [1]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import joblib
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
faq=pd.read_csv('faq.csv')
faq.head()

,id,question,answer
0,1,What is the best time to workout?,Morning workouts can boost energy and metaboli...
1,2,How much protein do I need daily?,0.8–1g protein per kg body weight is recommended.
2,3,Can I lose weight without cardio?,"Yes, strength training and calorie deficit can..."
3,4,What foods help muscle growth?,"Lean meats, fish, legumes, and protein-rich fo..."
4,5,How often should I work out weekly?,3–5 times per week is effective for most people.


In [6]:
EMBEDDED_MODEL = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMBEDDED_MODEL,device=device)

# prepare corpus
corpus = faq['question'].astype(str).tolist()
embeddings = embedder.encode(corpus, convert_to_numpy=True, show_progress_bar=True)

# save for future use
os.makedirs("models/chatbot",exist_ok=True)
np.save("models/chatbot/faq_embeddings.npy",embeddings)
joblib.dump(embedder,"models/chatbot/embedder.joblib")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['models/chatbot/embedder.joblib']

In [12]:
nn = NearestNeighbors(n_neighbors=1,metric='cosine').fit(embeddings)
def retrieve_nn(query, k=3):
  q_emb=embedder.encode([query], convert_to_numpy=True)
  dists, idxs=nn.kneighbors(q_emb,n_neighbors=k)
  # cosine distance -> simialrity = 1 - dist
  return [(idxs[0][i], 1-dists[0][i], faq.iloc[idxs[0][i]]['question'], faq.iloc[idxs[0][i]]['answer']) for i in range(len(idxs[0]))]

In [13]:
GEN_MODEL_NAME = "microsoft/DialoGPT-medium"  # or "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_NAME)
gen_model = AutoModelForCausalLM.from_pretrained(GEN_MODEL_NAME).to(device)

# simple generate function
def generate_response_generative(prompt, max_new_tokens=100, temperature=0.7, top_p=0.9, top_k=50):
    input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors="pt").to(device)
    out = gen_model.generate(
        input_ids,
        max_length=input_ids.shape[1] + max_new_tokens,
        do_sample=True,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
    )
    # decode only the generated part
    gen_text = tokenizer.decode(out[0, input_ids.shape[1]:], skip_special_tokens=True)
    return gen_text

# test
print(generate_response_generative("Hello! How do I lose weight?", max_new_tokens=40))

By eating less.


In [14]:
## rag based
def rag_generate(query, k=3, max_context_chars=1500):
    results = retrieve_nn(query, k=k)
    # join top-k answers/questions as context (truncate to avoid exceeding token limit)
    context = "\n\n".join([f"Q: {r[2]}\nA: {r[3]}" for r in results])
    # optionally truncate characters
    if len(context) > max_context_chars:
        context = context[:max_context_chars]
    prompt = f"Use the context to answer the question.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"
    return generate_response_generative(prompt)

# example
print(rag_generate("How much protein do I need?"))

I didn't want to get to personal and ask the question, but thanks for the reply.


In [18]:
RETRIEVAL_THRESHOLD = 0.65

def hybrid_retrieval_first(query, k=3, threshold=RETRIEVAL_THRESHOLD):
    results = retrieve_nn(query, k=k)
    top_idx, top_sim, q_text, ans_text = results[0]
    if top_sim >= threshold:
        return {"mode":"retrieval", "answer": ans_text, "score": float(top_sim)}
    else:
        # fallback to generative
        gen = generate_response_generative(query)
        return {"mode":"generative", "answer": gen, "score": None}

print(hybrid_retrieval_first("What is the best exercise for weight loss with a bmi of 25?")['answer'])

Cardio. Get a good core work out.
